# GridSearch y Pipelines

En este notebook vamos a ver un par de cosas que ya hemos mencionado en algún que otro notebook:
 - **GridSearch**: es una herramienta de optimización que se utiliza para buscar la mejor combinación de hiperparámetros. Consiste en definir una red de valores para cada parámetro, y el objeto se encargará de gestionar la combinación de cada uno de ellos, devolviendo el mejor resultado en cada caso.
 
 - **Pipelines**: son una interfaz para trabajar con combinaciones de objetos de ``sklearn`` como si fueran uno solo. De este modo, podremos diseñar combinaciones de tratamientos de datos con modelos y utilizarlos en nuestros GridSearch, o aplicarles cross validation, como vimos en el notebook pasado. En este caso los implementaremos directamente, en lugar de utilizar una función específica.
 
Para implementar la búsqueda de la combinación de parámetros óptima, vamos a ver 3 estrategias, que irán avanzando de menor a mayor complejidad:

### 1. Método simple

Itera un algoritmo sobre un conjunto de hiperparámetros. Para ello, utilizaremos el objeto ``GridSearchCV``:

In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, train_test_split


# Cargamos los datos:
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state=42)

# Creamos modelo:
svc = svm.SVC()

# Definimos parámetros:
parameters = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
    'gamma': ['scale', 'auto'],
    'coef0': [-10, -1, 0, 0.1, 0.5, 1, 10, 100]
    
}

# Nos creamos iterador basado en cross validation:
grid = GridSearchCV(estimator = svc,
                   param_grid = parameters,
                   n_jobs = -1,
                   scoring = 'accuracy',
                   cv = 10)

# Entrenamos sobre train:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
                         'coef0': [-10, -1, 0, 0.1, 0.5, 1, 10, 100],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [3]:
# Obtenemos los mejores parámetros:
print("Best estimator:", grid.best_estimator_)
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

Best estimator: SVC(C=0.1, coef0=-10, kernel='linear')
Best params: {'C': 0.1, 'coef0': -10, 'gamma': 'scale', 'kernel': 'linear'}
Best score: 0.9583333333333334


In [4]:
# Y lo probamos sobre test:
best_estimator = grid.best_estimator_
best_estimator.score(X_test, y_test)

1.0

Aquí lo estaríamos haciendo ya bien por fin, ya que dividimos entre train y test. Luego hacemos el cross validation sobre train. Y, finalmente, probamos el desempeño sobre test para verificar que no nos sale nada desorbitado.

### EJERCICIO

Vamos a poner esto en práctica con el último ejemplo que vimos para la selección de variables. Prueba a variar los parámetros partiendo de un algoritmo de regresión logística para predecir si una persona ha sobrevivido o no.

Para que no andes rebuscando en los notebooks, te dejo por aquí el tratamiento común que le habíamos dado el otro día, pero te recomiendo probar por ti mismo a cambiar cosas y crear nuevas variables también:

In [5]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df
# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')
# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())
# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)
# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Sex'
X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Sex']]
X = df[X_cols]
y = df[y_col]
# Tratamiento categóricas:
X = pd.get_dummies(X)

### 2. Forma pro


En este caso, además, introduciremos los pipelines, creándolos utilizando directamente su constructor ``Pipeline(steps=[])``. Si te fijas bien en el código, con este método podemos comparar diferentes algoritmos, con sus diferentes combinaciones de parámetros, gracias a estos pipelines. La sintáxis es el objeto ``Pipeline``, que recibe como parámetros ``steps``, que es una lista de tuplas, donde la primera se corresponde con el nombre que le das al paso, y el segundo el obejto que se ejecuta en ese paso:

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

pipe = Pipeline(steps=[('classifier', DecisionTreeClassifier())])

logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.arange(0, 4, 0.5)
}

decision_tree_params = {
    'classifier': [DecisionTreeClassifier()],
    'classifier__max_depth': [10, 8, 5, 2],
    'classifier__criterion': ['entropy', 'gini']
}

svc_params = {
    'classifier': [svm.SVC()],
    'classifier__kernel': ['linear', 'rbf', 'sigmoid']
}

search_space = [logistic_params, decision_tree_params, svc_params]

grid = GridSearchCV(pipe,
                   search_space,
                   cv=10,
                   n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression(C=1.5)],
                          'classifier__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['entropy', 'gini'],
                          'classifier__max_depth': [10, 8, 5, 2]},
                         {'classifier': [SVC()],
                          'classifier__kernel': ['linear', 'rbf', 'sigmoid']}])

In [6]:
print(grid.best_estimator_)
print(grid.best_params_)
print(grid.best_score_)

Pipeline(steps=[('classifier', LogisticRegression(C=1.5))])
{'classifier': LogisticRegression(C=1.5), 'classifier__C': 1.5, 'classifier__penalty': 'l2'}
0.9583333333333334


In [44]:
grid.score(X_test, y_test)

1.0

In [45]:
print(grid.predict(X_test))
print(y_test)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


### EJERCICIO

Bueno pues, para que podamos comparar qué nos ofrece una versión frente a otra, vamos a volver sobre el mismo ejemplo del Titanic para probar estas técnicas.

1. Utiliza el GridSearch para encontrar la combinación de parámetros que mejores resultados ofrece, probando, esta vez, con otros algoritmos como Árbol de decisión y SVC.

Utiliza la misma separación train/test de antes. También sería conveniente que utilizaras una semilla apra tener cierta reproducibilidad de los datos, es decir, que vuelva a salir lo mismo si vuelves a ejecutar:

In [5]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df
# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')
# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())
# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)
# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Sex'
X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Sex']]
X = df[X_cols]
y = df[y_col]
# Tratamiento categóricas:
X = pd.get_dummies(X)

### 3. Next Level

Finalmente, tenemos otra posible implementación, donde guardamos cada uno de los grids para tener un mayor control de cada modelo:

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [26]:
# Comenzamos definiendo los pipelines principales:
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("reglog", LogisticRegression())
])

svc = Pipeline([
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("svc", svm.SVC())
])

decision_tree = DecisionTreeClassifier()

# Y definimos sus parámetros:
re_log_param = {
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "reglog__penalty": ["l1", "l2"],
    "reglog__C": np.arange(0, 4, 0.5)
}

svc_param = {
    "selectkbest__k": [1, 2, 3],
    "svc__C": np.arange(0.1, 0.9, 0.1),
    "svc__kernel": ['linear', 'poly', 'rbf']
}

decision_tree_params = {
    'max_depth': [10, 100, 500, 1000],
    'criterion': ['gini', 'entropy']
}

# Nos creamos los grids de cada uno:
gs_reg_log = GridSearchCV(reg_log,
                         re_log_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

gs_svm = GridSearchCV(svc,
                         svc_param,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)

gs_decision_tree = GridSearchCV(decision_tree,
                         decision_tree_params,
                         cv = 10,
                         scoring='accuracy',
                         n_jobs=-1,
                         verbose=1)
grids = {
    "gs_reg_log": gs_reg_log,
    "gs_svm": gs_svm,
    "gs_rand_forest": gs_decision_tree
}
#gs_reg_log.fit(X_train, y_train)
#print("Best estimator:", gs_reg_log.best_estimator_)
#print("Best params:", gs_reg_log.best_params_)
#print("Best score:", gs_reg_log.best_score_)

In [27]:
for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 72 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s


Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [28]:
grids.

dict_items([('gs_reg_log', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'reglog__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]),
                         'reglog__penalty': ['l1', 'l2']},
             scoring='accuracy', verbose=1)), ('gs_svm', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('svc', SVC())]),
             n_jobs=-1,
             param_grid={'selectkbest__k': [1, 2, 3],
                         'svc__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
    

In [56]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns = ['Grid', 'Best score'])
best_grids.sort_values(by='Best score', ascending=False)

,Grid,Best score
1,gs_svm,0.950000
0,gs_reg_log,0.941667
2,gs_rand_forest,0.925000


In [57]:
print("Best estimator:", gs_svm.best_estimator_)
print("Best params:", gs_svm.best_params_)
print("Best score:", gs_svm.best_score_)

Best estimator: Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=2)),
                ('svc', SVC(C=0.1, kernel='linear'))])
Best params: {'selectkbest__k': 2, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Best score: 0.9499999999999998


In [70]:
# Obtenemos mejores resultados:
estimador = gs_svm.best_estimator_
estimador.score(X_test, y_test)

1.0

In [71]:
# Podemos predecir:
estimador.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [72]:
# Extraemos los nombres de las variables:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [62]:
# Sacamos los pvalores de cada uno
estimador['selectkbest'].pvalues_

array([1.72477507e-23, 2.69962606e-14, 1.93619072e-72, 3.57639330e-65])

Si queremos guardar el modelo, podemos ayudarnos de **pickle**:

In [63]:
import pickle

# Escribir
with open('finished_model.model', 'wb') as archivo_salida:
    pickle.dump(estimador, archivo_salida)

In [64]:
# Leer
with open('finished_model.model', 'rb') as archivo_entrada:
    pipeline_importado = pickle.load(archivo_entrada)

Una vez importado, ya podemos utilizarlo como si lo acabásemos de crear:

In [73]:
pipeline_importado

Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=2)),
                ('svc', SVC(C=0.1, kernel='linear'))])

In [74]:
new_flowers = np.array([[6.9, 3.1, 5.1, 2.3],
                        [5.8, 2.7, 3.9, 1.2]])

In [7]:
pipeline_importado.predict(X_test)
pipeline_importado.predict(new_flowers)

NameError: name 'pipeline_importado' is not defined

In [76]:
gs_svm.best_estimator_.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### EJERCICIO

Para terminar este ejemplo, y que se entienda de la mejor manera posible, vamos a repetir el experimento con el mismo dataset y el mismo objetivo. La única diferecia será que, esta vez, nos centraremos en añadir pasos a los pipelines principales (como selectores de variables o escalados).

También nos interesará saber cuál es la mejor combinación para cada modelo. Utiliza, al menos, 3 de estos modelos: LogisticRegression, DecisionTreeClasiffier, LinearSVC, SVC y KNN; y realiza la combinación variando, al menos, 2 parámetros diferentes para cada uno.

Además, finaliza guardando el modelo que mejor resultado te ofrezca para utilizarlo en cualquier otro momento.

Como hemos comentado antes, utiliza la misma separación train/test de antes. También sería conveniente que utilizaras una semilla.

In [5]:
import pandas as pd
df = pd.read_csv("../../../data/titanic.csv", sep='\t')
df
# El que queda de Embarked, lo vamos a rellenar con "S" porque es el mayoritario.
df['Embarked'] = df['Embarked'].fillna('S')
# En Age, vamos a completar los valores nulos con la media de los valores de Age.
df['Age'] = df['Age'].fillna(df['Age'].mean())
# En Pclass, vamos a convertirlo a string para luego hacer el One Hot Encoding.
df['Pclass'] = df['Pclass'].astype(str)
# Separamos las variables que podemos utilizar para predecir de la que queremos predecir:
y_col = 'Sex'
X_cols = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Sex']]
X = df[X_cols]
y = df[y_col]
# Tratamiento categóricas:
X = pd.get_dummies(X)